## Задачи сегментации фасадов и детекции объектов (окна, колонны)

#### Необходимо обучить собственный алгоритм tensorflow сегментации фасада главного здания, а также алгоритм определения числа окон и числа колонн.
#### Заказчик хотел бы выделять на фото фасад "главного" здания, если на изображении несколько зданий.

Решение задачи требует создания нескольких алгоритмов машинного обучения для решения трех различных задач:

* Сегментация фасада главного здания на изображении
* Определение числа окон на фасаде
* Определение числа колонн на фасаде

Для решения каждой из этих задач можно использовать различные алгоритмы машинного обучения, такие как нейронные сети и алгоритмы компьютерного зрения.

Например, для **сегментации фасада главного здания на изображении можно использовать алгоритмы семантической сегментации**, такие как FCN, U-Net или SegNet, которые могут обучаться на размеченных данных, где каждый пиксель на изображении помечен как принадлежащий к фасаду или нет.

Для **определения числа окон и числа колонн на фасаде можно использовать алгоритмы детектирования объектов**, такие как Faster R-CNN, RetinaNet или YOLO, которые могут обучаться на размеченных данных, где каждый объект (окно или колонна) на изображении помечен с помощью ограничивающей рамки.

Также можно использовать готовые модели машинного обучения, предварительно обученные на больших наборах данных, такие как ImageNet, и дообучать их на своих размеченных данных.

Однако, чтобы создать алгоритмы машинного обучения для решения этих задач, необходимо иметь доступ к размеченным данным, то есть набору изображений, на которых фасады главного здания помечены, а также размечены окна и колонны. Такие данные были получены на первом этапе предобработки данных путем ручной разметки изображений.

Кроме того, необходимо провести оценку качества полученных моделей машинного обучения на тестовых данных, чтобы убедиться в их эффективности и точности. Для этого можно использовать метрики, такие как точность, полноту и F1-меру, а также визуально сравнить результаты работы модели с оригинальными изображениями.

Кроме того, стоит учитывать, что **задача определения числа окон и колонн на фасаде может быть достаточно сложной, особенно если на изображении находятся объекты разных размеров, расположенные на разном расстоянии от камеры, в разных условиях освещения и т.д. Поэтому для получения точных результатов может потребоваться использование дополнительных методов предобработки изображений, таких как поворот, масштабирование и улучшение контрастности.**

*Для обучения моделей используем U-Net и YOLO, так как работаем без GPU-ускорителя и ограничены домашним ноутбуком и мощностями Google Colab.*


## 1) Сегментация фасадов с помощью U-Net

**U-Net - это нейронная сеть для семантической сегментации изображений, разработанная для медицинских изображений, но также успешно применяемая для сегментации фасадов зданий.**

Для проведения сегментации фасадов с помощью U-Net необходимо иметь размеченный набор данных, на котором обучится модель. Разметка данных представляет собой пометку каждого пикселя на изображении как принадлежащего к фасаду или не принадлежащего. Этот процесс может быть трудоемким и затратным.

#### Обучение модели включает в себя:

1. Нормализацию и предобработку данных, таких как изменение размера изображений, масштабирование значений пикселей и т.д.
2. Разбиение данных на обучающую, валидационную и тестовую выборки.
3. Определение архитектуры модели и настройка параметров, таких как количество слоев и фильтров, функция активации, оптимизатор и т.д.
4. Обучение модели на обучающей выборке с использованием алгоритма обратного распространения ошибки и минимизации функции потерь.
5. Оценка модели на валидационной выборке и настройка параметров модели для улучшения ее качества.
6. Тестирование модели на тестовой выборке для оценки ее точности и эффективности.

После того, как модель обучена, мы можем использовать ее для сегментации фасадов на новых изображениях. Для этого необходимо применить модель к каждому пикселю изображения и определить, принадлежит ли он фасаду или нет. Результаты сегментации могут быть сохранены в виде маски, где каждый пиксель помечен как принадлежащий фасаду или нет

In [524]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.layers import Conv2DTranspose
from keras import backend as K
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
import shutil
import os

import os
import numpy as np
import tensorflow as tf
import keras
from  tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.models import load_model
from matplotlib.pyplot import imsave,imshow
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
import random
from scipy import ndarray
from keras import backend as K
from keras.models import Model
from keras.layers import Input, BatchNormalization, Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, concatenate, Concatenate, UpSampling2D, Activation
from keras.losses import categorical_crossentropy
from keras.applications.inception_resnet_v2 import InceptionResNetV2, inception_resnet_block, conv2d_bn
from keras.applications.densenet import DenseNet121, dense_block, transition_block

from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate

In [525]:
label_png = pd.read_csv("C:\\Users\\HOME\\PycharmProjects\\CV_BuildingAnalytics\\labels_png.csv")
label_png.head()

,filename,background,facade,window,door,cornice,sill,balcony,blind,deco,molding,pillar,shop
0,cmp_b0001.png,0,1,1,1,1,1,1,1,1,1,1,1
1,cmp_b0002.png,1,0,1,1,1,1,1,1,1,1,1,1
2,cmp_b0003.png,1,1,0,1,1,1,1,1,1,1,1,1
3,cmp_b0004.png,1,1,1,0,1,1,1,1,1,1,1,1
4,cmp_b0005.png,1,1,1,1,0,1,1,1,1,1,1,1


In [526]:
labels_facade = label_png[['filename', 'facade']].copy()
labels_facade.head()

,filename,facade
0,cmp_b0001.png,1
1,cmp_b0002.png,0
2,cmp_b0003.png,1
3,cmp_b0004.png,1
4,cmp_b0005.png,1


In [527]:
labels_facade.to_csv('labels_facade.csv', index=False)

In [528]:
# Set the path to the directory containing the image data and the CSV file with labels
data_dir = "C:\\Users\\HOME\\PycharmProjects\\CV_BuildingAnalytics\\"
labels_file = "C:\\Users\\HOME\\PycharmProjects\\CV_BuildingAnalytics\\labels.csv"

labels_df = pd.read_csv(os.path.join(data_dir, labels_file))
labels_df

,filename,background,facade,window,door,cornice,sill,balcony,blind,deco,molding,pillar,shop
0,cmp_b0001.jpg,0,1,1,1,1,1,1,1,1,1,1,1
1,cmp_b0001.png,0,1,1,1,1,1,1,1,1,1,1,1
2,cmp_b0002.jpg,1,0,1,1,1,1,1,1,1,1,1,1
3,cmp_b0002.png,1,0,1,1,1,1,1,1,1,1,1,1
4,cmp_b0003.jpg,1,1,0,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,cmp_b0376.png,1,1,0,1,1,0,0,1,1,1,1,1
752,cmp_b0377.jpg,1,1,0,1,1,1,0,1,1,1,1,1
753,cmp_b0377.png,1,1,0,1,1,1,0,1,1,1,1,1
754,cmp_b0378.jpg,1,1,0,1,1,1,0,0,1,1,1,1


In [529]:
# split the data into training and validation sets
train_size = 0.6  # Proportion of data to use for training
val_size = 0.4  # Proportion of data to use for validation

sss = StratifiedShuffleSplit(n_splits=1, test_size=val_size, random_state=12345)
train_indices, val_indices = next(sss.split(X=labels_df['filename'], y=labels_df['facade']))

train_df = labels_df.iloc[train_indices]
val_df = labels_df.iloc[val_indices]

# Create subdirectories for the subsets
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'valid')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Copy the image files to the subdirectories
for _, row in train_df.iterrows():
    src_path = os.path.join(data_dir, 'base', row['filename'])
    dst_path = os.path.join(train_dir, row['filename'])
    shutil.copy(src_path, dst_path)

for _, row in val_df.iterrows():
    src_path = os.path.join(data_dir, 'base', row['filename'])
    dst_path = os.path.join(val_dir, row['filename'])
    shutil.copy(src_path, dst_path)

# Save the labels for the subsets to separate CSV files
train_df.to_csv(os.path.join(data_dir, 'train_labels.csv'), index=False)
val_df.to_csv(os.path.join(data_dir, 'valid_labels.csv'), index=False)

In [530]:
# class_names = list(train_generator.class_indices.keys())
# print('Названия классов из читаемой папки:', class_names)

In [531]:
MAIN_DIR = "C:\\Users\\HOME\\PycharmProjects\\CV_BuildingAnalytics\\"

TRAIN_DIR = "C:\\Users\\HOME\\PycharmProjects\\CV_BuildingAnalytics\\train"
VALID_DIR = "C:\\Users\\HOME\\PycharmProjects\\CV_BuildingAnalytics\valid"

TRAIN_LABELS = "C:\\Users\\HOME\\PycharmProjects\\CV_BuildingAnalytics\\train_labels.csv"
VALID_LABELS = "C:\\Users\\HOME\\PycharmProjects\\CV_BuildingAnalytics\\valid_labels.csv"

INPUT_SHAPE = (128, 128, 3)

In [626]:
def load_train(path):
    labels = pd.read_csv(path + 'train_labels.csv')
    labels['facade'] = labels['facade'].astype(str) # Convert facade column to string type
    train_datagen = ImageDataGenerator(validation_split=0.25, horizontal_flip=True, rescale=1./255)
    X_train = train_datagen.flow_from_dataframe(
        dataframe=labels,
        directory=path + '\\train',
        x_col='filename',
        y_col='facade',
        target_size=(128, 128),
        batch_size=16,
        class_mode='binary',
        subset='training',
        seed=12345)

    # Modify the labels to have shape (batch_size, 128, 128, 1)
    y_train = np.expand_dims(X_train.labels, axis=-1)
    y_train = np.tile(y_train, (1, 128, 128, 1))

    return X_train, y_train


def load_valid(path):
    labels = pd.read_csv(path + 'valid_labels.csv')
    labels['facade'] = labels['facade'].astype(str) # Convert facade column to string type
    test_datagen = ImageDataGenerator(validation_split=0.25, rescale=1./255)
    X_valid = test_datagen.flow_from_dataframe(
        dataframe=labels,
        directory=path + '\\valid',
        x_col='filename',
        y_col='facade',
        target_size=(128, 128),
        batch_size=16,
        class_mode='binary',
        subset='validation',
        seed=12345)

    # Modify the labels to have shape (batch_size, 128, 128, 1)
    y_valid = np.expand_dims(X_valid.labels, axis=-1)
    y_valid = np.tile(y_valid, (1, 128, 128, 1))

    return X_valid, y_valid

In [627]:
# def dice_coefficient_loss(y_true, y_pred):
#     axis = (1, 2)
#     numerator = 2.0 * tf.reduce_sum(y_true * y_pred, axis=axis)
#     denominator = tf.reduce_sum(y_true + y_pred, axis=axis)
#     return 1.0 - tf.reduce_mean(numerator / denominator)

def dice_coefficient(y_true, y_pred, smooth=1):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1 - (dice_coefficient(y_true, y_pred))

In [629]:
def create_model(input_shape):

    inputs = tf.keras.layers.Input(input_shape)
    s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

    conv1 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(s)
    conv1 = tf.keras.layers.Dropout(0.1)(conv1)
    conv1 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(conv1)
    pool1 = tf.keras.layers.MaxPooling2D((2, 2))(conv1)

    conv2 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(pool1)
    conv2 = tf.keras.layers.Dropout(0.1)(conv2)
    conv2 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(conv2)
    pool2 = tf.keras.layers.MaxPooling2D((2, 2))(conv2)

    conv3 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(pool2)
    conv3 = tf.keras.layers.Dropout(0.2)(conv3)
    conv3 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(conv3)
    pool3 = tf.keras.layers.MaxPooling2D((2, 2))(conv3)

    conv4 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(pool3)
    conv4 = tf.keras.layers.Dropout(0.2)(conv4)
    conv4 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(conv4)
    pool4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = tf.keras.layers.Conv2D(256, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(pool4)
    conv5 = tf.keras.layers.Dropout(0.3)(conv5)
    conv5 = tf.keras.layers.Conv2D(256, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(conv5)

    upconv6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv5)
    upconv6 = tf.keras.layers.concatenate([upconv6, conv4])
    conv6 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(upconv6)
    conv6 = tf.keras.layers.Dropout(0.2)(conv6)
    conv6 = tf.keras.layers.Conv2D(128, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(conv6)

    upconv7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv6)
    upconv7 = tf.keras.layers.concatenate([upconv7, conv3])
    conv7 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(upconv7)
    conv7 = tf.keras.layers.Dropout(0.2)(conv7)
    conv7 = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(conv7)

    upconv8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv7)
    upconv8 = tf.keras.layers.concatenate([upconv8, conv2])
    conv8 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(upconv8)
    conv8 = tf.keras.layers.Dropout(0.1)(conv8)
    conv8 = tf.keras.layers.Conv2D(32, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(conv8)

    upconv9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(conv8)
    upconv9 = tf.keras.layers.concatenate([upconv9, conv1], axis=3)
    conv9 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(upconv9)
    conv9 = tf.keras.layers.Dropout(0.1)(conv9)
    conv9 = tf.keras.layers.Conv2D(16, (3, 3), activation=tf.keras.activations.elu, kernel_initializer='he_normal',
                                padding='same')(conv9)

    outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
    optimizer = Adam(learning_rate=0.0001)

    model.compile(optimizer, loss='binary_crossentropy', metrics=[dice_coefficient,'accuracy'])

    return model

In [658]:
def train_model(model,
                X_train, y_train,
                X_valid, y_valid,
                batch_size=None,
                epochs=5,
                steps_per_epoch=None,
                validation_steps=None):

    if steps_per_epoch is None:
        steps_per_epoch = int(np.ceil(X_train.samples / batch_size))
    if validation_steps is None:
        validation_steps = int(np.ceil(X_valid.samples / batch_size))

    train_generator = ImageDataGenerator().flow(X_train, y_train, batch_size=batch_size)
    valid_generator = ImageDataGenerator().flow(X_valid, y_valid, batch_size=batch_size)

    callbacks = [EarlyStopping(patience=6, monitor='loss')]

    model.fit_generator(train_generator,
                        validation_data=valid_generator,
                        epochs=epochs,
                        steps_per_epoch=steps_per_epoch,
                        validation_steps=validation_steps,
                        verbose=2,
                        callbacks=callbacks)

    return model

In [659]:
X_train, y_train = load_train(MAIN_DIR)

Found 340 validated image filenames belonging to 2 classes.


In [660]:
X_valid, y_valid = load_valid(MAIN_DIR)

Found 75 validated image filenames belonging to 2 classes.


In [661]:
u_net = create_model(INPUT_SHAPE)
u_net

In [ ]:
history = train_model(u_net, X_train, y_train, X_valid, y_train, batch_size=16, epochs=10)

### Model 2

In [557]:
def create_model2(input_shape, weights='imagenet'):

    inputs = Input(shape=input_shape)
    # Contracting path (encoder)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)

    # Expanding path (decoder)
    up4 = Conv2DTranspose(128, 2, strides=(2, 2), activation='relu', padding='same')(conv3)
    merge4 = concatenate([conv2, up4], axis=3)
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(merge4)
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(conv4)

    up5 = Conv2DTranspose(64, 2, strides=(2, 2), activation='relu', padding='same')(conv4)
    merge5 = concatenate([conv1, up5], axis=3)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(merge5)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(conv5)

    # Output layer
    outputs = Conv2D(1, 1, activation='sigmoid')(conv5)

    # Compile the model
    optimizer = Adam(learning_rate=0.0001)
    model = Model(inputs=inputs, outputs=outputs)

    model.compile(optimizer=optimizer, loss='binary_crossentropy',
                  metrics=[dice_coefficient,'accuracy'])

    model.summary()

    model.save('facade_segmentation_v2.h5')

    return model

In [515]:
def train_model(model, train_data, valid_data, batch_size=None, epochs=5,
                steps_per_epoch=None, validation_steps=None):

    if steps_per_epoch is None:
        steps_per_epoch = len(train_data)
    if validation_steps is None:
        validation_steps = len(test_data)

    callbacks = [EarlyStopping(patience=6, monitor='val_loss')]

    model.fit(train_data,
              validation_data=valid_data,
              batch_size=batch_size,
              epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2,
              callbacks=callbacks)

    return model

In [385]:
model2 = create_model2((128, 128, 3))

Model: "model_47"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_64 (InputLayer)          [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_766 (Conv2D)            (None, 128, 128, 64  1792        ['input_64[0][0]']               
                                )                                                                 
                                                                                                  
 conv2d_767 (Conv2D)            (None, 128, 128, 64  36928       ['conv2d_766[0][0]']             
                                )                                                          

In [388]:
u_net1  = train_model(model2, train_data, test_data, batch_size=16, epochs=10)

Epoch 1/10


ValueError: in user code:

    File "C:\Users\HOME\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\HOME\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\HOME\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\HOME\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\HOME\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "C:\Users\HOME\AppData\Loc

In [235]:

# Manually calculate dice coefficient
dice_coefficients = []
for i in range(len(history.history['loss'])):
    dice_coeff = dice_coefficient(K.constant(test_data[i][1]), K.constant(u_net.predict(test_data[i][0])))
    dice_coefficients.append(dice_coeff)
print('Dice coefficient:', np.mean(dice_coefficients))


NameError: name 'history' is not defined